In [1]:
import numpy as np
import cv2
import argparse
import sys
import os
import utils.lib_images_io as lib_images_io
import utils.lib_plot as lib_plot
import utils.lib_commons as lib_commons
from utils.lib_openpose import SkeletonDetector
from utils.lib_tracker import Tracker
from utils.lib_tracker import Tracker
from utils.lib_classifier import ClassifierOnlineTest
from utils.lib_classifier import *  # Import all sklearn related libraries



# -- Command-line input


SRC_DATA_TYPE = "video"
SRC_DATA_PATH = "../cam1.mp4"
SRC_MODEL_PATH = "models/trained_classifier.pickle"

# DST_FOLDER_NAME = get_dst_folder_name(SRC_DATA_TYPE, SRC_DATA_PATH)

# -- Settings

cfg_all = lib_commons.read_yaml(ROOT + "config/config.yaml")
cfg = cfg_all["s5_test.py"]

CLASSES = np.array(cfg_all["classes"])
SKELETON_FILENAME_FORMAT = cfg_all["skeleton_filename_format"]

# Action recognition: number of frames used to extract features.
WINDOW_SIZE = int(cfg_all["features"]["window_size"])


# Video setttings

# If data_type is webcam, set the max frame rate.
SRC_WEBCAM_MAX_FPS = float(cfg["settings"]["source"]
                           ["webcam_max_framerate"])

# If data_type is video, set the sampling interval.
# For example, if it's 3, then the video will be read 3 times faster.
SRC_VIDEO_SAMPLE_INTERVAL = int(cfg["settings"]["source"]
                                ["video_sample_interval"])

# Openpose settings
OPENPOSE_MODEL = cfg["settings"]["openpose"]["model"]
OPENPOSE_IMG_SIZE = cfg["settings"]["openpose"]["img_size"]

# Display settings
img_disp_desired_rows = int(cfg["settings"]["display"]["desired_rows"])


# -- Function


def select_images_loader(src_data_type, src_data_path):
    if src_data_type == "video":
        images_loader = lib_images_io.ReadFromVideo(
            src_data_path,
            sample_interval=SRC_VIDEO_SAMPLE_INTERVAL)

    elif src_data_type == "folder":
        images_loader = lib_images_io.ReadFromFolder(
            folder_path=src_data_path)

    elif src_data_type == "webcam":
        if src_data_path == "":
            webcam_idx = 0
        elif src_data_path.isdigit():
            webcam_idx = int(src_data_path)
        else:
            webcam_idx = src_data_path
        images_loader = lib_images_io.ReadFromWebcam(
            SRC_WEBCAM_MAX_FPS, webcam_idx)
    return images_loader


class MultiPersonClassifier(object):
    ''' This is a wrapper around ClassifierOnlineTest
        for recognizing actions of multiple people.
    '''

    def __init__(self, model_path, classes):

        self.dict_id2clf = {}  # human id -> classifier of this person

        # Define a function for creating classifier for new people.
        self._create_classifier = lambda human_id: ClassifierOnlineTest(
            model_path, classes, WINDOW_SIZE, human_id)

    def classify(self, dict_id2skeleton):
        ''' Classify the action type of each skeleton in dict_id2skeleton '''

        # Clear people not in view
        old_ids = set(self.dict_id2clf)
        cur_ids = set(dict_id2skeleton)
        humans_not_in_view = list(old_ids - cur_ids)
        for human in humans_not_in_view:
            del self.dict_id2clf[human]

        # Predict each person's action
        id2label = {}
        for id, skeleton in dict_id2skeleton.items():

            if id not in self.dict_id2clf:  # add this new person
                self.dict_id2clf[id] = self._create_classifier(id)

            classifier = self.dict_id2clf[id]
            id2label[id] = classifier.predict(skeleton)  # predict label
            # print("\n\nPredicting label for human{}".format(id))
            # print("  skeleton: {}".format(skeleton))
            # print("  label: {}".format(id2label[id]))

        return id2label

    def get_classifier(self, id):
        ''' Get the classifier based on the person id.
        Arguments:
            id {int or "min"}
        '''
        if len(self.dict_id2clf) == 0:
            return None
        if id == 'min':
            id = min(self.dict_id2clf.keys())
        return self.dict_id2clf[id]


def remove_skeletons_with_few_joints(skeletons):
    ''' Remove bad skeletons before sending to the tracker '''
    good_skeletons = []
    for skeleton in skeletons:
        px = skeleton[2:2+13*2:2]
        py = skeleton[3:2+13*2:2]
        num_valid_joints = len([x for x in px if x != 0])
        num_leg_joints = len([x for x in px[-6:] if x != 0])
        total_size = max(py) - min(py)
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # IF JOINTS ARE MISSING, TRY CHANGING THESE VALUES:
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        if num_valid_joints >= 5 and total_size >= 0.1 and num_leg_joints >= 0:
            # add this skeleton only when all requirements are satisfied
            good_skeletons.append(skeleton)
    return good_skeletons


def draw_result_img(img_disp, ith_img, humans, dict_id2skeleton,
                    skeleton_detector, multiperson_classifier):
    ''' Draw skeletons, labels, and prediction scores onto image for display '''
    
    # Resize to a proper size for display
    r, c = img_disp.shape[0:2]
    desired_cols = int(1.0 * c * (img_disp_desired_rows / r))
    img_disp = cv2.resize(img_disp,
                          dsize=(1000, 600))

    # Draw all people's skeleton
    skeleton_detector.draw(img_disp, humans)

    # Draw bounding box and label of each person
    if len(dict_id2skeleton):
        for id, label in dict_id2label.items():
            skeleton = dict_id2skeleton[id]
            # scale the y data back to original
            skeleton[1::2] = skeleton[1::2] / scale_h
            # print("Drawing skeleton: ", dict_id2skeleton[id], "with label:", label, ".")
            
#             if(label == ''):
#                 if id in dictId_label:
#                     label = dictId_label[id]
#                     dict_id2label[id] = label
                
            lib_plot.draw_action_result(img_disp, id, skeleton, label)

    # Add blank to the left for displaying prediction scores of each class
    img_disp = lib_plot.add_white_region_to_left_of_image(img_disp)

    cv2.putText(img_disp, "Frame:" + str(ith_img),
                (20, 20), fontScale=1.5, fontFace=cv2.FONT_HERSHEY_PLAIN,
                color=(0, 0, 0), thickness=2)

    # Draw predicting score for only 1 person
    if len(dict_id2skeleton):
        classifier_of_a_person = multiperson_classifier.get_classifier(
            id='min')
        classifier_of_a_person.draw_scores_onto_image(img_disp)
    return img_disp


# -- Detector, tracker, classifier

skeleton_detector = SkeletonDetector(OPENPOSE_MODEL, OPENPOSE_IMG_SIZE)

multiperson_tracker = Tracker()

multiperson_classifier = MultiPersonClassifier(SRC_MODEL_PATH, CLASSES)

# -- Image reader and displayer
images_loader = select_images_loader(SRC_DATA_TYPE, SRC_DATA_PATH)
img_displayer = lib_images_io.ImageDisplayer()

dictid_label = {}

# -- Init output

# -- Read images and process
try:
    ith_img = -1
    while images_loader.has_image():

        # -- Read image
        img = images_loader.read_image()
        ith_img += 1
        img_disp = img.copy()
        print(f"\nProcessing {ith_img}th image ...")

        # -- Detect skeletons
        humans = skeleton_detector.detect(img)
        skeletons, scale_h = skeleton_detector.humans_to_skels_list(humans)
        skeletons = remove_skeletons_with_few_joints(skeletons)

        
        # -- Track people
        dict_id2skeleton = multiperson_tracker.track(
            skeletons)  # int id -> np.array() skeleton
        

        # -- Recognize action of each person
        if len(dict_id2skeleton):
            dict_id2label = multiperson_classifier.classify(
                dict_id2skeleton)

        # -- Draw
        img_disp = draw_result_img(img_disp, ith_img, humans, dict_id2skeleton,
                                   skeleton_detector, multiperson_classifier)

        # Print label of a person
        if len(dict_id2skeleton):
            min_id = min(dict_id2skeleton.keys())
            print("prediced label is :", dict_id2label[min_id])
            print("dict_id2label")
            print(dict_id2label)
            dictId_label = dict_id2label

        # -- Display image, and write to video.avi
        img_displayer.display(img_disp, wait_key_ms=1)
        
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            break

finally:
    cv2.destroyAllWindows()
    print("Program ends")

C:\Users\Haliva\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Haliva\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Haliva\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Haliva\anaconda3\envs\myenv\lib\site-packages\tensorflow\pyth

Instructions for updating:
Colocations handled automatically by placer.


[2021-02-23 20:21:44,280] [TfPoseEstimator] [DEBUG] inference+ original shape=1920x1080
[2021-02-23 20:21:44,280] [TfPoseEstimator] [DEBUG] inference+ original shape=1920x1080



Processing 0th image ...


[2021-02-23 20:21:45,108] [TfPoseEstimator] [DEBUG] inference- heatMat=328x184 pafMat=328x184
[2021-02-23 20:21:45,108] [TfPoseEstimator] [DEBUG] inference- heatMat=328x184 pafMat=328x184
[2021-02-23 20:21:45,129] [TfPoseEstimator] [DEBUG] estimate time=0.01895
[2021-02-23 20:21:45,129] [TfPoseEstimator] [DEBUG] estimate time=0.01895
[2021-02-23 20:21:45,135] [TfPoseEstimator] [INFO] inference image in 0.8549 seconds.
[2021-02-23 20:21:45,135] [TfPoseEstimator] [INFO] inference image in 0.8549 seconds.
[2021-02-23 20:21:45,300] [TfPoseEstimator] [DEBUG] inference+ original shape=1920x1080
[2021-02-23 20:21:45,300] [TfPoseEstimator] [DEBUG] inference+ original shape=1920x1080


...

ProcFtr.has_neck_and_thigh(x) True

Mean score:
 [1.15131550e-105 1.70192394e-058 5.00000000e-001 1.71311434e-034
 1.86354632e-030 4.99999915e-001 8.51060419e-008 1.22553443e-035
 3.46811982e-039]
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
prediced label is : 
dict_id2label
{20: '', 26: '', 27: '', 28: '', 29: ''}

Processing 14th image ...


[2021-02-23 20:21:55,613] [TfPoseEstimator] [DEBUG] inference- heatMat=328x184 pafMat=328x184
[2021-02-23 20:21:55,613] [TfPoseEstimator] [DEBUG] inference- heatMat=328x184 pafMat=328x184
[2021-02-23 20:21:55,627] [TfPoseEstimator] [DEBUG] estimate time=0.01297
[2021-02-23 20:21:55,627] [TfPoseEstimator] [DEBUG] estimate time=0.01297
[2021-02-23 20:21:55,636] [TfPoseEstimator] [INFO] inference image in 0.6593 seconds.
[2021-02-23 20:21:55,636] [TfPoseEstimator] [INFO] inference image in 0.6593 seconds.
[2021-02-23 20:21:55,758] [TfPoseEstimator] [DEBUG] inference+ original shape=1920x1080
[2021-02-23 20:21:55,758] [TfPoseEstimator] [DEBUG] inference+ original shape=1920x1080


ProcFtr.has_neck_and_thigh(x) True

Mean score:
 [8.94189162e-145 3.18561806e-047 1.78411550e-016 8.01276658e-046
 1.86354631e-030 4.99999915e-001 5.00000085e-001 1.22553459e-035
 3.53018445e-055]
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
prediced label is : kick
dict_id2label
{20: 'kick', 28: '', 29: '', 30: '', 31: ''}

Processing 15th image ...


[2021-02-23 20:21:56,379] [TfPoseEstimator] [DEBUG] inference- heatMat=328x184 pafMat=328x184
[2021-02-23 20:21:56,379] [TfPoseEstimator] [DEBUG] inference- heatMat=328x184 pafMat=328x184
[2021-02-23 20:21:56,396] [TfPoseEstimator] [DEBUG] estimate time=0.01539
[2021-02-23 20:21:56,396] [TfPoseEstimator] [DEBUG] estimate time=0.01539
[2021-02-23 20:21:56,401] [TfPoseEstimator] [INFO] inference image in 0.6429 seconds.
[2021-02-23 20:21:56,401] [TfPoseEstimator] [INFO] inference image in 0.6429 seconds.


ProcFtr.has_neck_and_thigh(x) True

Mean score:
 [1.09943532e-155 3.18561806e-047 3.12056598e-024 8.42972825e-032
 1.58196796e-075 1.42676442e-047 1.00000000e+000 1.64263919e-042
 1.52061906e-054]
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
ProcFtr.has_neck_and_thigh(x) True
prediced label is : kick
dict_id2label
{20: 'kick', 28: '', 31: '', 32: '', 33: ''}
Program ends
